In [1]:
import numpy as np

from pyquil.wavefunction import Wavefunction
# We'll use pyQuil's Wavefunction class to pretty print
def pprint(wvf : np.ndarray):
    print(Wavefunction(wvf))

We'll show that quantum mechanics supports this contradiction by giving a specific example of a system state and measurement configurations for Alice, Bob, Charlie, Debbie.

The system will be defined as an even superposition between the $|00\rangle$, $|01\rangle$, and $|10\rangle$ states.

In [2]:
sys = 1/np.sqrt(3) * np.array([1, 1, 1, 0])
pprint(sys)

(0.5773502692+0j)|00> + (0.5773502692+0j)|01> + (0.5773502692+0j)|10>


The measurement settings will be defined by observables in the usual computational and X basis.

In [3]:
zero = np.asarray([1, 0])
one = np.asarray([0, 1])
basis = [zero, one]
_ = [pprint(ee) for ee in basis]

(1+0j)|0>
(1+0j)|1>


In [10]:
# We can define the X basis by applying the Hadamard transform
hadamard = 1/np.sqrt(2) * np.asarray([[1, 1], [1, -1]])
xbasis = [hadamard@ee for ee in basis]
_ = [pprint(ee) for ee in xbasis]

(0.7071067812+0j)|0> + (0.7071067812+0j)|1>
(0.7071067812+0j)|0> + (-0.7071067812+0j)|1>


Now, for each of Alice and Bob, we will define observables that correspond to those bases. Recall from the scenario set up, that is we set x or y to 1 then Alice and Bob will open up their respective friend's laboratories and ask for the result of Charlie's (or Debbie's) computational basis measurement. This corresponds to looking at a computational measurement in the standard basis.

When x or y is instead set to 2, then Alice (or Bob) will make a measurement of their friend's laboratory in the X basis. In practice this could be achieved by doing a Hadamard gate on the joint system of the friend and the part of the system S that they have in their lab.

In [11]:
def observable(x):
    if x == 1:
        return basis
    if x == 2:
        return xbasis

Next we need a few helper functions to implement the standard Born rule from quantum mechanics. The `prob` function will now tell us the probability of an outcome given by a projector to be obtained from an underlying state.

In [7]:
def dagger(state : np.ndarray) -> np.ndarray:
    return np.conj(np.transpose(state))

def prob(proj, state):
    return dagger(state)@proj@state

We now have enough to calculate the probabilities for different bit outcomes for Alice (a) and Bob (b) under their respective measurement settings x and y.

In [12]:
def experiment(a, b, x, y):
    # measurement projector for Alice
    proj_A = np.outer(observable(x)[a], observable(x)[a])
    # measurement projector for Bob
    proj_B = np.outer(observable(y)[b], observable(y)[b])
    proj = np.kron(proj_A, proj_B)
    return prob(proj, sys)

For example we can look at the probability that Alice and Bob both get a 1 when they have both set their measurement settings to 2. As this is non-zero we would mark this as a possible outcome. Note that this is event $E_1$ that we need to be possible for a contradiction.

In [9]:
experiment(a=1, b=1, x=2, y=2)

0.08333333333333333

We can now check to see if events $E_2$, $E_3$, and $E_4$ are impossible (have probability 0). If they are impossible then we have all the needed events for the contradiction described in the previous section.

In [14]:
print("E_2 probability is: ", experiment(a=0, b=1, x=1, y=2))
print("E_3 probability is: ", experiment(a=1, b=0, x=2, y=1))
print("E_4 probability is: ", experiment(a=1, b=1, x=1, y=1))

E_2 probability is:  0.0
E_3 probability is:  0.0
E_4 probability is:  0.0


Thus quantum mechanics supports a Probabilistic Extended Wigner's Friend Scenario contradiction between the assumptions of Absoluteness of Observed Events and Probabilistic Local Agency.